## Problem 3: Learning a data-driven Linear Dynamical System (LDS)

### Part A:
Find the data associated with the jPCA paper (included as `exampleData.mat`). Load up
the data, and read the `NOTES.pdf` file to identify which data matrix corresponds to “condition
27”. What do the values in this matrix represent? Plot the data as a heatmap.


The `Notes.pdf` says that "each of the 108 elements is one 'condition': one reach type that our monkey performed.' This means that these are neural recordings from  

In [ ]:
from sthom215_problem_3_ex_3 import *
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

pwd = Path().cwd()
fname = 'exampleData.mat'
fpath = pwd / fname
COND = 27

mat, time_steps = loadData(COND, fpath)


In [ ]:
sns.heatmap(mat, fmt=".2f", cbar=True)

# Set the axis labels
plt.xlabel("Time bin (10ms)")
plt.ylabel("Neuron")

# Set the title of the plot
plt.title(f"Mean firing rate of Monkey Neurons reaching under condition {COND}")

# Show the plot
plt.show()

### Part B 
In this problem we will set up the optimization to learn a linear system guiding a
population of neurons. For one dataset write out the probabilistic model corresponding to the
discrete approximation of $\frac{dx_t}{dt} = Ax_t + \epsilon$ where $\epsilon$ is independently identically distributed
Gaussian noise with zero mean and standard deviation $\sigma^2I$. For the discretization, use the simple
approximation to the differential: $\frac{dx_t}{dt} \approx \frac{x_t - x_{t-1}}{dt}$. 

So,

$$ \partial x =\frac{x_t - x_{t-1}}{\Delta} = Ax_t + \epsilon$$

where $x_t$ and $x_{t-1}$ matrices over time $[1,t]$ and $[0,t-1]$ respectively.


Choose $\Delta$ small.

Derive the ML as:

$$\mathcal{L}(A) = p(\partial x|A) = \prod_{i=1}^n\mathcal{N}(x_{ij};\mu, \sigma^2)$$
$$\sum_{i=1}^n-\log\mathcal{N}(x_{ij};\mu, \sigma^2)$$



$$\sum_{i=1}^n \frac{x_{ij}-\mu}{\sigma^2}$$

* Now what is the same solution when given $k$ datasets that all follow the same dynamics? (Hint: think about each dataset as independent: what’s an important property of independent random variables with respect to their probabilities?).

Since each dataset is independent then the solutions to each of the datasets are $k$ independent solutions. Since the dynamics are assumed to be Markovian each time sequence is only dependent on its previous. It follows that we can simply take the product of each of the marices entry wise assuming that each element in the matrix is a probability. Through the log, this becomes a sum of each of their solutions up to some scaling of the contributions of each. This maintains the Markov assumption as well.


The derived likelihood represents $P(x|A)$. Use this to set up a ML estimate of
$A$. 

Code up in MATLAB the solution you derived and run on the jPCA dataset. Reconstruct
the data by taking each $x_t$ and predicting $x_{t+1}$ by taking one step given the dynamical system
you found (via the discrete equation you derived above). Compute the reconstruction error of
the data (norm of the difference between the estimated $x_{t+1}$ and the real $x_{t+1}$), and plot the
histogram of these errors across all time-points.

In [1]:
# run system forward one step
from sthom215_problem_3_ex_3 import *
from pathlib import Path
import numpy as np 
import numpy.linalg as la

pwd = Path().cwd()
fname = 'exampleData.mat'
fpath = pwd / fname

bin = 1

COND = 27
mat27, time_bins = loadData(COND, fpath)
# beginning changes in the system
x0, x1 = get_delayed_matrices(mat27)
dx = x1 - x0

In [18]:
A_lstsq = np.linalg.lstsq(x0.T, dx.T, rcond=None)[0]

In [19]:
from scipy.optimize import minimize 

def obj(A, x0, dx, gamma=0.01):
    _A = A.reshape(218,218)
    return la.norm(dx - _A @ x0, ord='fro') + gamma * la.norm(_A, ord='fro')
    

initA = np.random.randn(218,218)

results = minimize(obj, x0=initA.flatten(),  args=(x1, dx),
                   method='L-BFGS-b', 
                   options={'disp':True, 'maxiter':1000})

finalA = results.x.reshape(218,218)

KeyboardInterrupt: 

In [20]:
x1recon = finalA @ x0
x1reconlstsq = A_lstsq @ x0

In [21]:
recon_error = la.norm(x1recon - x1, ord='fro')
recon_error_lstsq = la.norm(x1reconlstsq - x1, ord='fro')
print(f'the reconstruction error is {recon_error}')
print(f'the reconstruction error using lstsq is {recon_error_lstsq}')

the reconstruction error is 13723.431606662261
the reconstruction error using lstsq is 14174.480519040679


In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the first plot
plt.figure(figsize=(10, 5))
sns.histplot(la.norm(x1recon - x1, ord=2, axis=1), label='BFGS')
plt.title('Histogram of BFGS Reconstruction Error')
plt.xlabel('Error')
plt.ylabel('Frequency')

sns.histplot(la.norm(x1reconlstsq - x1, ord=2, axis=1), label='Lstsq')
plt.title('Histogram of Lstsq Reconstruction Error')
plt.xlabel('Error')
plt.ylabel('Frequency')

plt.show()

## Part C
Repeat part B but consider only the top 6 principal components of each matrix $X$.
REMEMBER: PCA projection includes a mean subtraction step! This is very important in projecting the data! Compute again the reconstruction error of the data given the learned dynamical system (with a one-step prediction), and plot the histogram of errors.


In [30]:
from sklearn.decomposition import PCA

pwd = Path().cwd()
fname = 'exampleData.mat'
fpath = pwd / fname

bin = 1

COND = 27
mat, time_bins = loadData(COND, fpath)
# beginning changes in the system

num_components = 6
pca = PCA(n_components=num_components)
fit_mat = pca.fit_transform(mat)

x0, x1 = get_delayed_matrices(fit_mat)
dx = x1 - x0


In [34]:
A_lstsq = np.linalg.lstsq(x0.T, dx.T, rcond=None)[0]
recon =  A_lstsq @ x0
reconsterror = la.norm(x1 - recon, ord='fro')
print(f'the reconstruction error is {reconsterror}')

the reconstruction error is 1809.8618078104512


Text(0, 0.5, 'Frequency')

In [35]:
plt.figure(figsize=(10, 5))
sns.histplot(la.norm(recon - x1, ord=2, axis=1), label='BFGS')
plt.title('Histogram of BFGS Reconstruction Error')
plt.xlabel('Error')
plt.ylabel('Frequency')


## Part D
The learned dynamics should provide a projection of the data onto a given space. Take
one of the first 6 datapoints in each dataset in the jPCA datasets, and use the learned dynamics
from Part C to extrapolate the neural system forward in time. Plot the first two dynamical
principal dimensions of the projection: how do they look?


In [116]:
from scipy.io import loadmat

pwd = Path().cwd()
fname = 'exampleData.mat'
fpath = pwd / fname

bin = 1

mat = loadmat(fpath)['Data']
dat = []
for i in range(mat.shape[1]):
    dat.append(mat[0][i].item()[0])

dat = np.asarray(dat).T.astype(np.int64)


In [121]:
first_six = dat[:,:6,:].transpose(2,0,1)
first_six.shape

(108, 218, 6)

In [143]:
future_states = list()

for i in range(1, 10):
    future_states.append(np.apply_along_axis(np.matmul, 1, first_six, np.power(A_lstsq, i)))

# Project future states back to the original space

In [144]:
future = np.concatenate(np.asarray(future_states),axis=2)
future = np.c_[first_six, future]

In [145]:
from sthom215_problem_2_ex_2 import create_SpikeTrain

ceil = np.ceil(future).astype(np.int64)
ceil[ceil < 0] = 0
st = create_SpikeTrain(ceil)

In [146]:
from elephant.gpfa import GPFA
import quantities as pq

bin_size = 1 * pq.ms
latent_dim = 2


gpfa_2dim = GPFA(bin_size=bin_size, x_dim=latent_dim)
all_trajectories= gpfa_2dim.fit_transform(st)

ValueError: Observation covariance matrix is rank deficient.
Possible causes: repeated units, not enough observations.

I am unable to plot these dimensinos because the extrapolated dynamics across the trials is rank insufficient. 

## Part E
Now run the jPCA algorithm on the same data. Look at the code and notes provided
by the authors and find the right lines of code to run on the dataset. Find the learned dynamics
and project the data forward as with Part D. Use “subplot” to plot the projected neural activity
next to each other. What key differences do you see between the dynamics? What differences
between the dynamical system matrices do you see? Are there similarities or differences e.g., in
the eigenspectra?



In [147]:
from sthom215_problem_2_ex_2 import create_SpikeTrain
from scipy.io import loadmat

pwd = Path().cwd()
fname = 'exampleData.mat'
fpath = pwd / fname

bin = 1

mat = loadmat(fpath)['Data']
dat = []
for i in range(mat.shape[1]):
    dat.append(mat[0][i].item()[0])

dat = np.asarray(dat).T.astype(np.int64)

st = create_SpikeTrain(dat)

In [148]:
from elephant.gpfa import GPFA
import quantities as pq

bin_size = 1 * pq.ms
latent_dim = 2


gpfa_2dim = GPFA(bin_size=bin_size, x_dim=latent_dim)
all_trajectories= gpfa_2dim.fit_transform(st)

Initializing parameters using factor analysis...

Fitting GPFA model...


In [149]:
# Code from https://elephant.readthedocs.io/en/latest/tutorials/gpfa.html
linewidth_single_trial = 0.5
color_single_trial = 'C0'
alpha_single_trial = 0.5

linewidth_trial_average = 2
color_trial_average = 'C1'


f, ax1 = plt.subplots(1, 1, figsize=(15, 5))

ax1.set_title('Latent dynamics extracted by GPFA')
ax1.set_xlabel('Dim 1')
ax1.set_ylabel('Dim 2')
ax1.set_aspect(1)
for single_trial_trajectory in all_trajectories:
    ax1.plot(single_trial_trajectory[0], single_trial_trajectory[1], '-', lw=linewidth_single_trial, c=color_single_trial, alpha=alpha_single_trial)
average_trajectory = np.mean(all_trajectories, axis=0)
ax1.plot(average_trajectory[0], average_trajectory[1], '-', lw=linewidth_trial_average, c=color_trial_average, label='Trial averaged trajectory')
ax1.legend()